<a href="https://colab.research.google.com/github/sMastrodimitris/A-Comparative-Study-On-Recommender-System-Approaches/blob/main/LLM_Retreiver_(flan_t5_large).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Install required packages
!pip install langchain langchain_text_splitters langchain_community chromadb sentence-transformers transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 36.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.4/188.4 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 3.7 MB/s eta 0:00:

In [1]:
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline
from langchain.llms import HuggingFacePipeline
import textwrap
import zipfile

In [2]:
# Extract ChromaDB
with zipfile.ZipFile("chroma_db_backup.zip", 'r') as zip_ref:
    zip_ref.extractall("chroma_db")

# Initialize embedding model
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
print("Initialized embedding model")

# Load the ChromaDB
vectordb = Chroma(
    persist_directory="chroma_db",
    embedding_function=embeddings
)
print("Loaded ChromaDB from disk")

<ipython-input-2-4d9c1ae35c2d>:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

Initialized embedding model


<ipython-input-2-4d9c1ae35c2d>:10: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  vectordb = Chroma(


Loaded ChromaDB from disk


In [36]:
model_name = "google/flan-t5-large"  # or try 'declare-lab/flan-alpaca-base'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=1024)
llm = HuggingFacePipeline(pipeline=pipe)
print(f"Initialized LLM ({model_name})")

# Simpler prompt template
template = """
Context:
{context}

Question:
{question}

Answer in five sentences using context.

Answer:
"""

PROMPT = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

Device set to use cpu


Initialized LLM (google/flan-t5-large)


In [39]:
# Create retriever (keep k small)
retriever = vectordb.as_retriever(search_kwargs={"k": 1})

# Use simple 'stuff' chain
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True,
    chain_type_kwargs={"prompt": PROMPT}
)

def format_docs(docs):
    formatted = []
    for i, doc in enumerate(docs):
        title = doc.metadata.get('title', 'Unknown')
        source = doc.metadata.get('source', 'Unknown')
        formatted.append(f"Document {i+1}:")
        formatted.append(f"Title: {title}")
        formatted.append(f"Source: {source}")
        formatted.append(f"Content: {textwrap.fill(doc.page_content, width=80)}")
        formatted.append("-" * 50)
    return "\n".join(formatted)

def query_rag_system(query):
    print(f"\nQuery: {query}")
    print("-" * 100)
    result = qa_chain({"query": query})
    answer = result["result"]
    source_docs = result["source_documents"]
    print("Answer:")
    print(textwrap.fill(answer, width=100))
    print("\nSource Documents:")
    print(format_docs(source_docs))
    print("=" * 100)
    return answer, source_docs

In [40]:
queries = [
    "What is stained glass?",
    "Legacy of Donatello",
    "man is the measure of all things"
]

for query in queries:
    query_rag_system(query)

def interactive_rag():
    print("\n" + "=" * 100)
    print("Interactive Art RAG System")
    print("Type 'exit' to quit")
    print("=" * 100)
    while True:
        query = input("\nEnter your question: ")
        if query.lower() == 'exit':
            break
        query_rag_system(query)


Query: What is stained glass?
----------------------------------------------------------------------------------------------------
Answer:
A product already have been invented in the Middle Ages

Source Documents:
Document 1:
Title: Rose window
Source: https://en.wikipedia.org/wiki/Rose_window
Content: A product already have been invented in the Middle Ages, stained glass only had
appeared in the rose window at the Abbey Church of Saint-Denis. However, it
started to become more popular around the earlier part of the 1200s, often the
money for the glass, being donated by the wealthy. The glass had a tenancy to be
dark and rich with color. The most common color combination was blue and red
color patterns. Bar tracery style
--------------------------------------------------

Query: Legacy of Donatello
----------------------------------------------------------------------------------------------------
Answer:
Donatello was a great artist and a great person.

Source Documents:
Document 1:
